In [72]:
# importing necessary libraries
import pandas as pd
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pylab as plt
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline
from pathlib import Path
import prettytable
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
from scipy.stats.mstats import gmean
%pylab inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
import glob 
warnings.simplefilter(action='ignore')
pio.renderers.default = 'iframe'
import ipywidgets as widgets
pyo.init_notebook_mode(connected = True)
from ipywidgets import interactive, HBox, VBox, widgets, interact
from ipywidgets import FloatSlider

Populating the interactive namespace from numpy and matplotlib


In [73]:
def summary_metrics(R,rf_annual=0.01, show=False):

    pt = prettytable.PrettyTable(['metric', 'value'])

    avg = np.mean(R)
    std_dev = np.std(R)
    rf = rf_annual / 252
    count = len(R)
    sharpe = (avg) / std_dev * np.sqrt(252)

    avg_annualized_return = (((1 + avg) ** 252) - 1)
    geo_mean = gmean(R + 1) - 1
    annualized_std = std_dev * np.sqrt(252)

    cumulative_end_return = np.cumsum(R).tail(1).values

    win = len(R[R >= 0])
    loss = len(R[R < 0])
    win_loss = win / loss
    win_ratio = win / (loss + win)

    rpt = {'mean': round(avg,4),
#            'avg_annualized_ret': avg_annualized_return,
#            'geometric_mean': geo_mean,
#            'cum_end_return': cumulative_end_return,
           'std_dev': std_dev,
#            'annualized_std': annualized_std,
           'Sharpe_ratio': round(sharpe,4),
#            'skewness': R.skew(),
#            'kurtosis': R.kurtosis(),
           'drawdown': get_drawdown(R),
#            'win_loss': win_loss,
           'win_ratio': round(win_ratio,4),
           'count': count
           }
    for (k, v) in rpt.items():
        pt.add_row([k, v])

    if show:
        print(pt)

    return rpt

In [74]:
def get_drawdown(R):
    # Calculated the peak
    peak = R.cummax()
    # Calculated difference of daily returns from the peak
    daily_drawdown = np.where(R!=peak,R-peak,0)
    # Found the maximum drop
    drawdown_percent = abs(daily_drawdown)/peak*100
    return (round(drawdown_percent.max(),4))

In [75]:
def compare_low(rpt_his_low,rpt_hat_low,rpt_lr_low):
    table_low = prettytable.PrettyTable(['metric', 'covar_his_low', 'covar_MGARCH_low', 'covar_LR_low'])
    for (k, v) in rpt_his_low.items():
        row = [k, v, rpt_hat_low[k],rpt_lr_low[k]]
        table_low.add_row(row)
    print(table_low)
    return table_low

def compare_med(rpt_his_med,rpt_hat_med,rpt_lr_med):
    table_med = prettytable.PrettyTable(['metric', 'covar_his_med', 'covar_MGARCH_med','covar_LR_med'])
    for (k, v) in rpt_his_med.items():
        row = [k, v, rpt_hat_med[k],rpt_lr_med[k]]
        table_med.add_row(row)
    print(table_med)
    return table_med

def compare_high(rpt_his_high,rpt_hat_high,rpt_lr_high):
    table_high = prettytable.PrettyTable(['metric', 'covar_his_high', 'covar_MGARCH_high','covar_LR_high'])
    for (k, v) in rpt_his_high.items():
        row = [k, v, rpt_hat_high[k],rpt_lr_high[k]]
        table_high.add_row(row)
    print(table_high)
    return table_high

In [76]:
filenametab = []
for filename in glob.glob('../outputdata/Stage2/Returns/*.csv'):
    filenametab.append(filename)
filenametab.sort(reverse=True)

returns = pd.read_csv(filenametab[0], parse_dates=True)
returns['date']= pd.to_datetime(returns.date)
returns.set_index('date', inplace=True)

In [77]:
# filenametab[0]

In [78]:
initial_investment = 100.00

In [79]:
def apply_cumsum(returns_series):
    cumsumret = (returns_series.add(1).cumprod()) * initial_investment
    cumsumret.iat[0] = initial_investment
    return cumsumret

In [80]:
returns['ret_low_his_cum'] = apply_cumsum(returns.ret_low_his)
returns['ret_low_hat_cum'] = apply_cumsum(returns.ret_low_hat)
returns['ret_low_lr_cum'] = apply_cumsum(returns.ret_low_lr)

returns['ret_med_his_cum'] = apply_cumsum(returns.ret_med_his)
returns['ret_med_hat_cum'] = apply_cumsum(returns.ret_med_hat)
returns['ret_med_lr_cum'] = apply_cumsum(returns.ret_med_lr)

returns['ret_high_his_cum'] = apply_cumsum(returns.ret_high_his)
returns['ret_high_hat_cum'] = apply_cumsum(returns.ret_high_hat)
returns['ret_high_lr_cum'] = apply_cumsum(returns.ret_high_lr)

## Comparing results for low risk level – 0.003 for a 252 Days look back at actual return values

In [81]:
compare_low(summary_metrics(returns.ret_low_his),summary_metrics(returns.ret_low_hat),summary_metrics(returns.ret_low_lr));

+--------------+-----------------------+----------------------+----------------------+
|    metric    |     covar_his_low     |   covar_MGARCH_low   |     covar_LR_low     |
+--------------+-----------------------+----------------------+----------------------+
|     mean     |         0.0001        |        0.0001        |        0.0001        |
|   std_dev    | 0.0038697594527083434 | 0.004782316745815615 | 0.003557650431954714 |
| Sharpe_ratio |         0.5046        |        0.2778        |        0.5058        |
|   drawdown   |        304.1274       |       310.0732       |       288.0784       |
|  win_ratio   |         0.5292        |        0.5236        |        0.5337        |
|    count     |          2479         |         2479         |         2479         |
+--------------+-----------------------+----------------------+----------------------+


## Comparing cummulative Dollar Return from for low risk level – 0.003 for a 252 Days look back at actual return values

In [82]:
# final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt = returns
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
    go.Scatter(x=filtered_df.index, y = filtered_df.ret_low_hat_cum,mode = 'lines',name = 'Low Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_low_his_cum,mode = 'lines',name = 'Low Risk Historical Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_low_lr_cum,mode = 'lines',name = 'Low Risk Liner Regression Return'),    
#     go.Scatter(x=filtered_df.index, y = np.cumsum(filtered_df.ret_low_hat),mode = 'lines',name = 'Low Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_low_his),mode = 'lines',name = 'Low Risk Historical Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.equity_daily_ret),mode = 'lines',name = 'Daily Equity Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.bond_daily_ret),mode = 'lines',name = 'Daily Bond Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.gold_daily_ret),mode = 'lines',name = 'Daily Gold Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.forex_daily_ret),mode = 'lines',name = 'Daily Forex Return'),
    ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'Low Risk Portfolio MGARCH/Historical/LR Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…

## Comparing results for Medium risk level – 0.005 for a 252 Days look back at actual return values

In [83]:
compare_med(summary_metrics(returns.ret_med_his),summary_metrics(returns.ret_med_hat),summary_metrics(returns.ret_med_lr));

+--------------+----------------------+----------------------+----------------------+
|    metric    |    covar_his_med     |   covar_MGARCH_med   |     covar_LR_med     |
+--------------+----------------------+----------------------+----------------------+
|     mean     |        0.0002        |        0.0001        |        0.0002        |
|   std_dev    | 0.006271153610778827 | 0.007857385704252708 | 0.006023019874247706 |
| Sharpe_ratio |        0.4388        |        0.1873        |        0.4206        |
|   drawdown   |       324.2333       |       327.6176       |        289.21        |
|  win_ratio   |        0.5345        |        0.5296        |        0.5333        |
|    count     |         2479         |         2479         |         2479         |
+--------------+----------------------+----------------------+----------------------+


## Comparing cummulative Dollar Return from for Medium risk level – 0.005 for a 252 Days look back at actual return values

In [84]:
final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
       
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_med_hat),mode = 'lines',name = 'Medium Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_med_his),mode = 'lines',name = 'Medium Risk Historical Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_med_hat_cum,mode = 'lines',name = 'Medium Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_med_his_cum,mode = 'lines',name = 'Medium Risk Historical Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_med_lr_cum,mode = 'lines',name = 'Medium Risk Linear Regression Return')

    ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'Medium Risk Portfolio MGARCH/Historical/LR Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…

## Comparing results for High risk level – 0.007 for a 252 Days look back at actual return values

In [85]:
compare_high(summary_metrics(returns.ret_high_his),summary_metrics(returns.ret_high_hat),summary_metrics(returns.ret_high_lr));

+--------------+----------------------+----------------------+----------------------+
|    metric    |    covar_his_high    |  covar_MGARCH_high   |    covar_LR_high     |
+--------------+----------------------+----------------------+----------------------+
|     mean     |        0.0002        |        0.0001        |        0.0002        |
|   std_dev    | 0.008208600419745752 | 0.009287279289386125 | 0.008498831995854054 |
| Sharpe_ratio |        0.3273        |        0.1709        |        0.3523        |
|   drawdown   |       296.4989       |       346.5857       |       293.0705       |
|  win_ratio   |        0.5341        |        0.5333        |        0.5377        |
|    count     |         2479         |         2479         |         2479         |
+--------------+----------------------+----------------------+----------------------+


## Comparing cummulative Dollar Return from for high risk level – 0.007 for a 252 Days look back at actual return values

In [86]:
final_csv_output_plt = returns[[col for col in returns.columns if col.startswith('ret_')]]
final_csv_output_plt['year'] = returns.index.year
xmin = final_csv_output_plt.year.unique().min()
xmax = final_csv_output_plt.year.unique().max()
slider = widgets.FloatRangeSlider(
    min=xmin,
    max=xmax,
    step=1,
    readout=True,
    readout_format='d',
    orientation='horizontal',
    description='Year')
slider.layout.width = '900px'
def update_plot(y):
    filtered_df = final_csv_output_plt.query( 'year>= ' + str(y[0]) +'and year<= ' + str(y[1]) )
    pyo.iplot(go.Figure(data=[
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_high_hat),mode = 'lines',name = 'High Risk MGARCH Return'),
#     go.Scatter(x=filtered_df.index,y = np.cumsum(filtered_df.ret_high_his),mode = 'lines',name = 'High Risk Historical Return')
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_high_hat_cum,mode = 'lines',name = 'High Risk MGARCH Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_high_his_cum,mode = 'lines',name = 'High Risk Historical Return'),
    go.Scatter(x=filtered_df.index,y = filtered_df.ret_high_lr_cum,mode = 'lines',name = 'High Risk Linear Regression Return')
   ],
    layout=go.Layout(xaxis = dict(title = 'Year'), yaxis = dict(title = 'Return in USD'),title = 'High Risk Portfolio MGARCH/Historical/LR Return from Year ' + str(int(y[0])) + ' To ' + str(int(y[1])))))
widgets.interact(update_plot, y=slider, continuous_update=False);

interactive(children=(FloatRangeSlider(value=(2012.5, 2017.5), description='Year', layout=Layout(width='900px'…